In [1]:
import numpy as np
import arviz as az
# import pymc3 as pm
import pickle
import matplotlib.pyplot as plt
from google.protobuf import text_format
from scipy.stats import multivariate_normal as mvn
from scipy.stats import skewnorm
from scipy.stats import norm
from scipy.interpolate import griddata
import pp_mix.protos.py.params_pb2 as params_pb2

from pp_mix.interface import ConditionalMCMC, cluster_estimate
from pp_mix.utils import loadChains, to_numpy, to_proto
from pp_mix.protos.py.state_pb2 import MultivariateMixtureState, EigenVector, EigenMatrix
from pp_mix.protos.py.params_pb2 import Params

In [ ]:
import pp_mix.protos.py.params_pb2 as params_pb2

from pp_mix.interface import ConditionalMCMC, cluster_estimate
from pp_mix.utils import loadChains, to_numpy, to_proto
from pp_mix.protos.py.state_pb2 import MultivariateMixtureState, EigenVector, EigenMatrix
from pp_mix.protos.py.params_pb2 import Params

In [2]:
params_file_1 = "pp_mix/resources/sampler_params_1.asciipb"


In [3]:
def generate_etas(mus, deltas_cov, cluster_alloc):
    out = np.vstack([[mvn.rvs(mean = mus[i,:], cov = deltas_cov[i]) for i in cluster_alloc]])
    return out

In [4]:
def generate_data(Lambda, etas, sigma_bar_cov):
    means = np.matmul(Lambda,etas.T)
    sigma_bar_cov_mat = np.diag(sigma_bar_cov)
    out = np.vstack([[mvn.rvs(mean = means[:,i], cov = sigma_bar_cov_mat) for i in range(etas.shape[0])]])
    return out

In [5]:
p=50
d=5
# 4 groups

Lambda=np.zeros((50,d))
Lambda[0:10,0]=np.ones(10)
Lambda[10:20,1]=np.ones(10)
Lambda[20:30,2]=np.ones(10)
Lambda[30:40,3]=np.ones(10)
Lambda[40:50,4]=np.ones(10)


mus = np.array([[-5.,10., 0., 5., 0.],
               [0.,-5.,10.,-5., 0.],
               [5.,-5.,0.,0.,-5.],
               [0.,0.,-10.,0.,5.]])

deltas_prec = deltas_cov = list([np.eye(5), np.eye(5), np.eye(5), np.eye(5)])

sigma_bar_prec = np.repeat(2, p)

sigma_bar_cov = 1/sigma_bar_prec

cluster_alloc = np.repeat(np.array([0,1,2,3]), 100)

etas = generate_etas(mus, deltas_cov, cluster_alloc)

data = generate_data(Lambda, etas, sigma_bar_cov)

#np.savetxt('data/data_d5.csv',data,delimiter=',')

In [6]:
data = np.loadtxt("data/data_d5.csv", delimiter=",")
ranges = np.array([[-50,-50],[50,50]])

In [14]:
#sampler0 = ConditionalMCMC(params_file = params_file_1)


In [15]:
#sampler0.run(10000, 40000, 40000, 10, data, ranges, 200)
#sampler0.serialize_chains("data/chains_sim_d5.recordio")


Trick, iter # 200  /  10000
Trick, iter # 400  /  10000
Trick, iter # 600  /  10000
Trick, iter # 800  /  10000
Trick, iter # 1000  /  10000
Trick, iter # 1200  /  10000
Trick, iter # 1400  /  10000
Trick, iter # 1600  /  10000
Trick, iter # 1800  /  10000
Trick, iter # 2000  /  10000
Trick, iter # 2200  /  10000
Trick, iter # 2400  /  10000
Trick, iter # 2600  /  10000
Trick, iter # 2800  /  10000
Trick, iter # 3000  /  10000
Trick, iter # 3200  /  10000
Trick, iter # 3400  /  10000
Trick, iter # 3600  /  10000
Trick, iter # 3800  /  10000
Trick, iter # 4000  /  10000
Trick, iter # 4200  /  10000
Trick, iter # 4400  /  10000
Trick, iter # 4600  /  10000
Trick, iter # 4800  /  10000
Trick, iter # 5000  /  10000
Trick, iter # 5200  /  10000
Trick, iter # 5400  /  10000
Trick, iter # 5600  /  10000
Trick, iter # 5800  /  10000
Trick, iter # 6000  /  10000
Trick, iter # 6200  /  10000
Trick, iter # 6400  /  10000
Trick, iter # 6600  /  10000
Trick, iter # 6800  /  10000
Trick, iter # 7000

Running, iter # 4600  /  40000
Running, iter # 4800  /  40000
Running, iter # 5000  /  40000
Running, iter # 5200  /  40000
Running, iter # 5400  /  40000
Running, iter # 5600  /  40000
Running, iter # 5800  /  40000
Running, iter # 6000  /  40000
Running, iter # 6200  /  40000
Running, iter # 6400  /  40000
Running, iter # 6600  /  40000
Running, iter # 6800  /  40000
Running, iter # 7000  /  40000
Running, iter # 7200  /  40000
Running, iter # 7400  /  40000
Running, iter # 7600  /  40000
Running, iter # 7800  /  40000
Running, iter # 8000  /  40000
Running, iter # 8200  /  40000
Running, iter # 8400  /  40000
Running, iter # 8600  /  40000
Running, iter # 8800  /  40000
Running, iter # 9000  /  40000
Running, iter # 9200  /  40000
Running, iter # 9400  /  40000
Running, iter # 9600  /  40000
Running, iter # 9800  /  40000
Running, iter # 10000  /  40000
Running, iter # 10200  /  40000
Running, iter # 10400  /  40000
Running, iter # 10600  /  40000
Running, iter # 10800  /  40000
Run

In [16]:
chain0 = loadChains("data/chains_sim_d5.recordio", MultivariateMixtureState)


In [17]:
ma_chain0 = np.array([len(x.a_means) for x in chain0])
mna_chain0 = np.array([len(x.na_means) for x in chain0])

print(ma_chain0.mean())
print(mna_chain0.mean())

4.0
0.487


In [18]:
s0_ca = np.array([x.clus_alloc for x in chain0])
best_cluster_s0 = cluster_estimate(s0_ca)
print(best_cluster_s0)

[3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.